# Recognizing digits with Layer

[![Open in Layer](https://app.layer.ai/assets/badge.svg)](https://app.layer.ai/layer/recognizing-digits) [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/layerai/examples/blob/main/recognizing-digits/recognizing-digits.ipynb) [![Layer Examples Github](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com/layerai/examples/tree/main/recognizing-digits)


In this project, we'll build a model to recognize digits. As part of the project, we'll cover the following: 

- Saving models in Layer. 
- Logging the model's accuracy. 
- Logging images.

## Layer installation

Ensure that you are using the latest version of [Layer](https://docs.app.layer.ai/docs/installation). 

In [ ]:
!pip install --upgrade layer-sdk -q
!pip install -U scikit-learn

## Register and Login

To start using Layer, you have to register and login. Run the following cell, click the link, register and paste the code in the input

In [ ]:
import layer
layer.login()

## Inititialize a  Layer project

The next step is to initialize a Layer project. All entities created thereafter will be stored under this project.  

In [ ]:
layer.init("recognizing-digits")

## Train Model


In this project, we'll load the dataset directly using Sklearn. The next step is to define a model function that returns a [model object](https://docs.app.layer.ai/docs/models/supported-frameworks). 

Saving the model to Layer is done by adding the [@model](https://docs.app.layer.ai/docs/sdk-library/model-decorator) decorator to the defined function. 


In this case, we perfoerm the following operations in the model function:

- Load the dataset using Sklearn.
- Log the digits to Layer using [Matplotlib](https://docs.app.layer.ai/docs/sdk-library/layer-log#charts). 
- Fit the images to a [Support Vector Machine](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) model. 
- Log the model's accuracy.
-Log the model's confusion matrix.

In [9]:
from layer.decorators import dataset, model,resources
@model("digit-recognizer")
def train_model():
    import matplotlib.pyplot as plt
    from sklearn import datasets, svm, metrics
    from sklearn.model_selection import train_test_split
    import pandas as pd
    import numpy as np

    digits = datasets.load_digits()

    _, axes = plt.subplots(nrows=2, ncols=4, figsize=(10, 10))
    for ax, image, label in zip(axes.flat, digits.images, digits.target):
        ax.set_axis_off()
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        ax.set_title("Training: %i" % label)
    
    layer.log({"Training Digits": plt})
    
    # flatten the images
    n_samples = len(digits.images)
    data = digits.images.reshape((n_samples, -1))

    # Create a classifier: a support vector classifier
    clf = svm.SVC(gamma=0.001)

    # Split data into 50% train and 50% test subsets
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.5, shuffle=False
    )

    # Learn the digits on the train subset
    clf.fit(X_train, y_train)

    # Predict the value of the digit on the test subset
    predicted = clf.predict(X_test)
    
    _, axes = plt.subplots(nrows=2, ncols=4, figsize=(10, 10))
    for ax, image, prediction in zip(axes.flat, X_test, predicted):
        ax.set_axis_off()
        image = image.reshape(8, 8)
        ax.imshow(image, cmap=plt.cm.gray_r, interpolation="nearest")
        ax.set_title(f"Prediction: {prediction}")
    
    layer.log({"Predictions": plt})
        
    prediction_metrics = metrics.classification_report(y_test, predicted,output_dict=True)
    layer.log({"Accuracy": prediction_metrics['accuracy']})

    disp = metrics.ConfusionMatrixDisplay.from_predictions(y_test, predicted)
    disp.figure_.suptitle("Confusion Matrix")
    layer.log({"Confusion Matrix Plot": disp.figure_})

    return clf

You can run the model training function in two ways: 

- On Layer infra by passing the function to the `layer.run` command.
- Locally by calling the function normally. 

Running the function locally utilizes your local compute but saves and versions the resulting model to Layer. Executing the model on Layer is useful when dealing with large data and models. 

In [ ]:
train_model()

Once the model trains successfully, Layer will print a link to the model. Clicking the link will show you all the logged entities, including; images, primitives and charts. 

## Where to go from here?

Now that you have created this project, you can:

- Join our [Slack Community ](https://bit.ly/layercommunityslack)
- Visit [Layer Examples Repo](https://github.com/layerai/examples) for more examples
- Browse [Trending Layer Projects](https://layer.ai) on our mainpage
- Check out [Layer Documentation](https://docs.layer.ai) to learn more